In [1]:
from working_with_files import *
from preprocessing import *
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [2]:
def sort_by_date(df):
    df1 = pd.DataFrame() # < 2008
    df2 = pd.DataFrame() # 2008 - 2013
    df3 = pd.DataFrame() # 2013 - 2016
    df4 = pd.DataFrame() # > 2016
    date1 = datetime.strptime('15-07-2008', '%d-%m-%Y').date()
    date2 = datetime.strptime('20-02-2013', '%d-%m-%Y').date()
    date3 = datetime.strptime('30-11-2016', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        try:
            date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
        except ValueError:
            date = datetime.strptime(row['date'], '%m-%Y').date()
        if date <= date1:
            df1 = df1.append(row, ignore_index=True)
        if date > date1 and date <= date2:
            df2 = df2.append(row, ignore_index=True)
        if date > date2 and date <= date3:
            df3 = df3.append(row, ignore_index=True)
        if date > date3:
            df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print()

def get_topics(documents, n_topics, n_top_words, models):
    
    random_seed = 42
    
    if 'nmf' in models:
        tfidf = tfidf_vectorizer.fit_transform(documents)
        tfidf_feature_names = tfidf_vectorizer.get_feature_names()
        
        nmf = NMF(n_components=n_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
        
        print('NMF')
        display_topics(nmf, tfidf_feature_names, n_top_words)
        
    if 'lda' in models:
        tf = tf_vectorizer.fit_transform(documents)
        tf_feature_names = tf_vectorizer.get_feature_names()
        
        lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5, learning_method='online', learning_offset=50., 
                                random_state=random_seed).fit(tf)
        
        print('LDA')
        display_topics(lda, tf_feature_names, n_top_words)

In [3]:
# read df and split it into 4 separate df4 based on publication date
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
sorted_dfs = sort_by_date(df)

In [4]:
# get unnecessary symbols to remove them from texts
symbols_to_del = generate_symbols_to_del(get_all_texts(df))
symbols_to_del.extend(list('abcdefghijklmnopqrstuvwxyz'))
print(symbols_to_del)

['%', '°', '>', ']', '№', '\n', '−', ':', '…', '\xad', '́', '-', '_', '{', '·', '"', "'", '#', '!', '–', '[', '”', '“', '―', '6', '3', '€', '8', '7', '@', '«', '*', '1', '$', '4', ';', '‑', '+', '/', '<', '´', '‒', ',', '5', '„', '`', '&', ')', '»', '‘', '(', '.', '－', '?', '=', '9', '’', '2', '！', '—', '0', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
for df in sorted_dfs:
    df['text_clean'] = df['text_lemmatized'].apply(clean, args=(symbols_to_del,))

In [45]:
# get and preprocess stopwords
# source: ruscorpora.ru

stopwords_raw = get_lexicon_from_file('russian_stopwords.txt')
stopwords = []
stopwords_string = ''.join(stopwords_raw)
strings = stopwords_string.split('\t')
for string in strings:
    if string.isalpha():
        stopwords.append(string)

# lemmatize stopwords

stopwords_lemmatized = [morph.parse(word)[0].normal_form for word in stopwords]
stopwords_lemmatized = list(set(stopwords_lemmatized))
stopwords = [stopword for stopword in stopwords_lemmatized if stopword.isalpha()]

stopwords.extend(['наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'между', 'год', 'вопрос', 'страна', 
                 'россия', 'российский', 'такой', 'государство', 'другой', 'также', 'рф', 'мид', 'сегодня', 'два', 'катерина',
                 'спецкор', 'дать', 'прокомментировать', 'интервью', 'федерация', 'лавров', 'сторона', 'российско', 'господин',
                 'рассказывать', 'министр', 'встреча', 'состояться', 'сайт', 'итог', 'официальный', 'глава', 'иностранный',
                 'дело', 'москва', 'александр', 'курировать', 'иванов', 'делегации', 'заместитель', 'комментировать', 'уважаемый',
                 'президент', 'москве', 'государственный' 'лавр', 'москвы', 'принять', 'участие', 'мероприятие', 'россии', 
                  'медведев', 'пресс', 'посетить'])

print(stopwords)

['ли', 'тут', 'себя', 'или', 'но', 'ни', 'который', 'вы', 'кто', 'вот', 'а', 'бы', 'как', 'мы', 'уже', 'потому', 'один', 'чем', 'она', 'тем', 'без', 'чтобы', 'потом', 'только', 'он', 'человек', 'даже', 'к', 'этот', 'быть', 'о', 'в', 'под', 'время', 'надо', 'когда', 'что', 'если', 'то', 'при', 'более', 'это', 'ничто', 'теперь', 'и', 'от', 'здесь', 'с', 'очень', 'есть', 'на', 'тогда', 'где', 'можно', 'по', 'из', 'весь', 'же', 'так', 'тот', 'нет', 'после', 'сказать', 'через', 'они', 'ты', 'за', 'я', 'для', 'мочь', 'раз', 'у', 'там', 'не', 'до', 'ещё', 'ну', 'наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'между', 'год', 'вопрос', 'страна', 'россия', 'российский', 'такой', 'государство', 'другой', 'также', 'рф', 'мид', 'сегодня', 'два', 'катерина', 'спецкор', 'дать', 'прокомментировать', 'интервью', 'федерация', 'лавров', 'сторона', 'российско', 'господин', 'рассказывать', 'министр', 'встреча', 'состояться', 'сайт', 'итог', 'официальный', 'глава', 'иностранный', 'дело

In [46]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=10, stop_words=stopwords)
tf_vectorizer = CountVectorizer(max_df=0.9, min_df=10, stop_words=stopwords)

In [71]:
get_topics(sorted_dfs[0].loc[:, 'text_clean'], 32, 15, ['nmf']) # до 2008

NMF
Topic 0:
международный развитие мировой важный новый экономический решение мир работа интерес сотрудничество стать глобальный внешний являться
Topic 1:
палестинский израильский израиль пна дорожный палестинец карты газа аббас ближневосточный палестино мирный посредник хамас квартета
Topic 2:
обсудить переговоры сотрудничество визит двусторонний договориться торгово отношение взаимодействие экономический конкретный ход развитие рамка сфера
Topic 3:
православный церковь русский религиозный святой общество церкви духовный религий цивилизаций нравственный дипломатия московский единство мира
Topic 4:
обсе смид организация организации безопасности реформа безопасность председательство словенский председатель обзор деятельность действующий документ полевой
Topic 5:
бдипч бюро наблюдатель наблюдение обсе электоральный мониторинговый избирательный выборы выбор мониторинг миссия получение обращать мониторинга
Topic 6:
русский соотечественник язык соотечественников рубежом русскоязычный конгр

In [80]:
get_topics(sorted_dfs[1].loc[:, 'text_clean'], 36, 15, ['nmf']) # 2008 - 2013

NMF
Topic 0:
международный экономический важный развитие глобальный мировой работа новый сотрудничество основа отношение стать решение интерес всего
Topic 1:
оон резолюция сб совет безопасность ливия генеральный международный миссия ливии член судан ливийский санкционный решение
Topic 2:
визит переговоры двусторонний обсудить сотрудничество республика ход взаимодействие рабочий торгово актуальный развитие отношений экономический мнение
Topic 3:
южный грузинский осетия абхазия грузия грузии тбилиси осетии югоосетинский абхазский саакашвили республика абхазии закавказье август
Topic 4:
русский язык центр культура языка фонд русскоязычный православный россотрудничество фестиваль культурный рубежом открытие литература школа
Topic 5:
сша американский вашингтон про клинтон госдепартамент госсекретарь снв обама ребёнок администрация государственный приёмный штат президентский
Topic 6:
одкб коллективный заседание совет организация безопасность сессия договор организации смид член рассмотреть б

In [74]:
get_topics(sorted_dfs[2].loc[:, 'text_clean'], 36, 15, ['nmf']) # 2013 - 2016

NMF
Topic 0:
международный развитие экономический важный интерес сотрудничество работа общий регион мировой основа усилие политический решение том
Topic 1:
сирийский сирия сирии алеппо боевик вооружённый террорист оппозиция джабхата террористический район ан правительство женевский оппозиции
Topic 2:
украинский украина киев украины юго киевский власть украине восток минский миссия донецкий представитель расследование заявление
Topic 3:
выздоровление погибших соболезнование выражать осуждать пострадавшим террористический решительно скорый родной желать близкие погибнуть теракт результат
Topic 4:
визит двусторонний сотрудничество переговоры обсудить ход актуальный республика повестка торгово взаимодействие лавр рабочий мнение международный
Topic 5:
сша американский вашингтон дж керри администрация госсекретарь госдепартамент против про санкция том санкционный обама заявление
Topic 6:
снг содружество заседание проект содружества участник совет смид внешнеполитический сотрудничество незави

In [88]:
get_topics(sorted_dfs[3].loc[:, 'text_clean'], 37, 15, ['nmf']) # С 2016

NMF
Topic 0:
международный экономический сотрудничество важный развитие интерес работа число оон рамка всего основа отношение взаимодействие регион
Topic 1:
сша американский вашингтон американец госдепартамент санкция против администрация какой санкционный собственный никакой попытка отношение подобный
Topic 2:
сирийский сирия сирии террорист гуманитарный провинция зона боевик район вооружённый алеппо мирный оон игил военный
Topic 3:
украина украинский киев украины киевский украине власть минский закон порошенко крым действие миссия донбасс язык
Topic 4:
двусторонний визит переговоры республика обсудить актуальный региональный рабочий мнение повестка лавр обмен сотрудничество ход международный
Topic 5:
советский память памятник исторический война великий чешский мировой отечественный войны история мемориальный освобождение победа красный
Topic 6:
химический озхий применение озхо оружие муфс расследование запрещение доклад сирийский хан оружия смр эксперт сирия
Topic 7:
молодая форум ди